In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.10.9`

import $ivy.$                                   

import $ivy.$                               

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [3]:
import scala.collection.Map
import org.apache.spark.sql._
import org.apache.spark.sql.{functions => f}
import org.apache.spark.sql.expressions.Window

val spark = NotebookSparkSession.builder().master("local[*]").getOrCreate()

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import scala.collection.Map

import org.apache.spark.sql._

import org.apache.spark.sql.{functions => f}

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@2191f8c3

In [4]:
val dataFrame = spark.read.option("header", "true").csv("data/tripadvisor_hotel_reviews.csv")

csv at cmd3.sc:1

1 / 1

dataFrame: DataFrame = [Review: string, Rating: string]

In [5]:
dataFrame.show(5)

show at cmd4.sc:1

1 / 1

+--------------------+------+
|              Review|Rating|
+--------------------+------+
|nice hotel expens...|     4|
|ok nothing specia...|     2|
|nice rooms not 4*...|     3|
|unique, great sta...|     5|
|great stay great ...|     5|
+--------------------+------+
only showing top 5 rows



In [6]:
val docCount = dataFrame.count()

count at cmd5.sc:1

4 / 4

count at cmd5.sc:1

1 / 1

docCount: Long = 20491L

In [7]:
val udfReplaceMultSpaces = f.udf((s: String) => s.trim.replaceAll("\\s+", " "))
val rowWindow = Window.partitionBy().orderBy("tokens")
val preprocessedTokens = 
    dataFrame.select("Review")
             .withColumn("Review", f.regexp_replace(f.col("Review"), "[^a-z0-9 ]+", ""))
             .withColumn("Review", udfReplaceMultSpaces(f.col("Review")))
             .withColumn("tokens", f.split(f.col("Review"), " "))
             .withColumn("doc_id", f.row_number().over(rowWindow))
             .select("doc_id", "tokens")
             .cache()

udfReplaceMultSpaces: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd6$Helper$$Lambda$4923/771050600@3d7095ce,
  StringType,
  Some(List(StringType))
)
rowWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@722521b9
preprocessedTokens: Dataset[Row] = [doc_id: int, tokens: array<string>]

In [8]:
preprocessedTokens.show()

show at cmd7.sc:1

4 / 4

show at cmd7.sc:1

1 / 1

+------+--------------------+
|doc_id|              tokens|
+------+--------------------+
|     1|[1, best, hotels,...|
|     2|[1, best, small, ...|
|     3|[1, choice, flore...|
|     4|[1, day, short, s...|
|     5|[1, spot, locatio...|
|     6|[1, star, best, l...|
|     7|[1, star, paid, 1...|
|     8|[1, star, resort,...|
|     9|[1, week, fantast...|
|    10|[1, word, wow, de...|
|    11|[1, year, wedding...|
|    12|[10, points, 10, ...|
|    13|[100, dollars, ni...|
|    14|[100, happy, sad,...|
|    15|[100, location, f...|
|    16|[100, perfect, hu...|
|    17|[100, star, ratin...|
|    18|[100, stays, 18, ...|
|    19|[1010, bahia, eff...|
|    20|[1010, impressed,...|
+------+--------------------+
only showing top 20 rows



In [9]:
val allDocs = (for {i <- 1 until docCount.toInt} yield i).toArray

allDocs: Array[Int] = Array(
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
...

In [10]:
val tokenCntByDocId = 
    preprocessedTokens
        .withColumn("tokens", f.explode(f.col("tokens")))
        .select("doc_id", "tokens").groupBy("doc_id").agg(f.count(f.col("tokens")).alias("token_cnt"))

tokenCntByDocId: DataFrame = [doc_id: int, token_cnt: bigint]

In [11]:
val termFrequency = 
    preprocessedTokens
        .withColumn("tokens", f.explode(f.col("tokens")))
        .select("doc_id", "tokens")
        .groupBy("doc_id", "tokens").agg(f.count("*").alias("cnt"))
        .join(tokenCntByDocId, "doc_id")
        .withColumn("tf", f.col("cnt") / f.col("token_cnt"))
        .select("doc_id", "tokens", "tf")

termFrequency: DataFrame = [doc_id: int, tokens: string ... 1 more field]

## Using sklearn-like IDF formula

In [12]:
val inverseDocumentFrequency = 
    preprocessedTokens.withColumn("tokens", f.explode(f.col("tokens")))
                      .select("doc_id", "tokens").groupBy("tokens")
                      .agg(f.count("doc_id").cast("long").alias("cnt"))
                      .withColumn("doc_cnt", f.lit(docCount))
                      .withColumn("idf", f.log((f.col("doc_cnt") + 1) / (f.col("cnt") + 1)) + 1)
                      .select("tokens", "idf")

inverseDocumentFrequency: DataFrame = [tokens: string, idf: double]

In [13]:
val top100InverseDocumentFrequency = inverseDocumentFrequency.sort(f.col("idf").desc).limit(100)

top100InverseDocumentFrequency: Dataset[Row] = [tokens: string, idf: double]

In [14]:
top100InverseDocumentFrequency.show()

show at cmd13.sc:1

1 / 1

show at cmd13.sc:1

1 / 1

+--------------+------------------+
|        tokens|               idf|
+--------------+------------------+
|    strategies|10.234642664499148|
|      catamera|10.234642664499148|
|       leaflet|10.234642664499148|
|wonderfulthank|10.234642664499148|
|         lippi|10.234642664499148|
|        faired|10.234642664499148|
|           45e|10.234642664499148|
|       baggles|10.234642664499148|
|  realtionship|10.234642664499148|
|        camery|10.234642664499148|
|         purmy|10.234642664499148|
|         tipso|10.234642664499148|
|         calss|10.234642664499148|
|         16400|10.234642664499148|
|         thand|10.234642664499148|
|      grandmas|10.234642664499148|
|    adequateas|10.234642664499148|
|   julyarrived|10.234642664499148|
|           178|10.234642664499148|
|   deskarrived|10.234642664499148|
+--------------+------------------+
only showing top 20 rows



In [15]:
val tfIdf = 
    top100InverseDocumentFrequency.join(termFrequency, "tokens")
                                  .withColumn("tf-idf", f.col("tf") * f.col("idf"))
                                  .select(f.col("doc_id"), f.col("tokens").alias("token"), f.col("tf-idf"))

tfIdf: DataFrame = [doc_id: int, token: string ... 1 more field]

In [16]:
tfIdf.show()

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd15.sc:1

1 / 1

+------+---------------+--------------------+
|doc_id|          token|              tf-idf|
+------+---------------+--------------------+
|     1|         1010my|   0.136461902193322|
|     1| wonderfulthank|   0.136461902193322|
|     5|          20800| 0.09841002562018412|
|     5|         acsses| 0.09841002562018412|
|     5|        baggles| 0.09841002562018412|
|     5|         camery| 0.09841002562018412|
|     7|       datednot| 0.17057737774165246|
|     7|          tipso| 0.17057737774165246|
|     7|            161| 0.17057737774165246|
|     7|          16400| 0.17057737774165246|
|     7|    sleepingnot| 0.17057737774165246|
|     8|         faired| 0.10133309568811037|
|     8|       grandmas| 0.10133309568811037|
|     9|    julyarrived|0.026108782307395783|
|     9|        soltour|0.026108782307395783|
|     9|    deskarrived|0.026108782307395783|
|     9|serviceactually|0.026108782307395783|
|     9|         usuall|0.026108782307395783|
|     9|      altlantic|0.02610878

In [17]:
val pivotedTfIdf = tfIdf.groupBy("doc_id").pivot("token").agg(f.sum("tf-idf")).na.fill(0)

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

pivot at cmd16.sc:1

1 / 1

pivotedTfIdf: DataFrame = [doc_id: int, 1010my: double ... 99 more fields]

In [18]:
pivotedTfIdf.show()

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd17.sc:1

...

|     7|              0.0|                0.0|    0.0|                 0.0|0.17057737774165246|0.17057737774165246|                0.0|                0.0|                0.0|                 0.0|  0.0|                0.0|                 0.0|                0.0|                 0.0|                0.0|                0.0|                0.0|                0.0|      0.0|                 0.0|                0.0|                0.0|                0.0|                 0.0|     0.0|                0.0|                0.0|                 0.0|                 0.0|0.17057737774165246|                 0.0|                 0.0|             0.0|                0.0|                 0.0|                 0.0|                0.0|                0.0|                 0.0|                0.0|                0.0|   0.0|                0.0|      0.0|                0.0|                0.0|                 0.0|       0.0|                 0.0|     0.0|                 0.0|                 0.0|          

|    11|              0.0|                0.0|    0.0|0.054730709435824317|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|  0.0|                0.0|                 0.0|                0.0|                 0.0|                0.0|                0.0|                0.0|                0.0|      0.0|                 0.0|                0.0|                0.0|                0.0|                 0.0|     0.0|                0.0|                0.0|                 0.0|                 0.0|                0.0|                 0.0|                 0.0|             0.0|                0.0|                 0.0|                 0.0|                0.0|                0.0|                 0.0|                0.0|                0.0|   0.0|                0.0|      0.0|                0.0|                0.0|                 0.0|       0.0|                 0.0|     0.0|                 0.0|                 0.0|          

|    19|              0.0|                0.0|    0.0|                 0.0|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|  0.0|                0.0|0.018507491255875492|                0.0|                 0.0|                0.0|                0.0|                0.0|                0.0|      0.0|                 0.0|                0.0|                0.0|                0.0|                 0.0|     0.0|                0.0|                0.0|                 0.0|0.018507491255875492|                0.0|                 0.0|                 0.0|             0.0|                0.0|                 0.0|0.018507491255875492|                0.0|                0.0|                 0.0|                0.0|                0.0|   0.0|                0.0|      0.0|                0.0|                0.0|                 0.0|       0.0|                 0.0|     0.0|                 0.0|                 0.0|0.01850749

|    23|              0.0|                0.0|    0.0|                 0.0|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|  0.0|                0.0|                 0.0|                0.0|                 0.0|                0.0|                0.0|                0.0|                0.0|      0.0|                 0.0|                0.0|                0.0|                0.0|                 0.0|     0.0|                0.0|                0.0|                 0.0|                 0.0|                0.0|                 0.0|                 0.0|             0.0|                0.0|                 0.0|                 0.0|                0.0|                0.0|                 0.0|                0.0|                0.0|   0.0|                0.0|      0.0|                0.0|                0.0|                 0.0|       0.0|                 0.0|     0.0|                 0.0|                 0.0|          

|    28|              0.0|                0.0|    0.0|                 0.0|                0.0|                0.0|                0.0|                0.0|                0.0|                 0.0|  0.0|                0.0|                 0.0|                0.0|                 0.0|                0.0|0.23260551510225339|                0.0|                0.0|      0.0|                 0.0|                0.0|                0.0|                0.0|                 0.0|     0.0|                0.0|0.23260551510225339|                 0.0|                 0.0|                0.0|                 0.0|                 0.0|             0.0|                0.0|                 0.0|                 0.0|                0.0|                0.0|                 0.0|                0.0|                0.0|   0.0|                0.0|      0.0|                0.0|                0.0|                 0.0|       0.0|                 0.0|     0.0|                 0.0|                 0.0|          

In [19]:
pivotedTfIdf.columns.length

res18: Int = 101